In [1]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import gensim.downloader as api

from preprocessing.utils import is_sentence_in_boundaries
from datasets_utils import get_luxury_data, get_tech_data, get_retail_data, get_big_basket_data
from preprocess import preprocess, with_category_features
from utils import (
    tfidf_vectorize,
    w2v_vectorize,
    display_metrics,
    accuracy_ml_score,
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /home/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/stepan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stepan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
device

'cpu'

In [3]:
luxury_data = get_luxury_data()
tech_data = get_tech_data()
retail_data = get_retail_data()
big_basket_data = get_big_basket_data()

# datasets = [big_basket_data]
# dataset_names = ['Big basket']
datasets = [big_basket_data, retail_data, luxury_data, tech_data]
dataset_names = ['Big basket', 'Retail', 'Luxury', 'Tech']

# Get datasets with description column preprocessed
tech_data['description'] = tech_data['description'].apply(preprocess)
luxury_data['description'] = luxury_data['description'].apply(preprocess)
retail_data['description'] = retail_data['description'].apply(preprocess)
big_basket_data['description'] = big_basket_data['description'].apply(preprocess)

# Preprocess categories
tech_data = with_category_features(tech_data)
luxury_data = with_category_features(luxury_data)
retail_data = with_category_features(retail_data)
big_basket_data = with_category_features(big_basket_data)

big_basket_data = big_basket_data[big_basket_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]
retail_data = retail_data[retail_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=250))]
luxury_data = luxury_data[luxury_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=100))]
tech_data = tech_data[tech_data['description'].apply(lambda x: is_sentence_in_boundaries(x, max_tokens=200))]

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[category] = data['category'].apply(lambda x: 1 if category in x else 0)
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/src/preprocess.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is us

In [4]:
big_basket_X_train, big_basket_X_test, big_basket_y_train, big_basket_y_test = train_test_split(
    big_basket_data['description'],
    big_basket_data[[column for column in big_basket_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13,
)

### TF-IDF

In [5]:
big_basket_X_train_tfidf, big_basket_X_test_tfidf = tfidf_vectorize(
    train_data=big_basket_X_train,
    test_data=big_basket_X_test,
)

In [9]:
ml_knn_tfidf_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

ml_knn_tfidf_grid_cv.fit(big_basket_X_train_tfidf, big_basket_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  29.8s
[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  30.9s
[CV 3/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  31.1s
[CV 1/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=  30.4s
[CV 2/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=  30.3s
[CV 3/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=  30.8s
[CV 1/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=  29.9s
[CV 2/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=  30.7s
[CV 3/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=  30.0s
[CV 1/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=  32.3s
[CV 2/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=  30.5s
[CV 3/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=  31.2s
[CV 1/3] END .............k=

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [10]:
ml_knn_tfidf_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [11]:
ml_knn_tfidf_best = MLkNN(**ml_knn_tfidf_grid_cv.best_params_)
ml_knn_tfidf_best.fit(big_basket_X_train_tfidf, big_basket_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [12]:
big_basket_y_pred_tfidf = ml_knn_tfidf_best.predict(big_basket_X_test_tfidf)

In [13]:
display_metrics(big_basket_y_test.to_numpy(), np.array(big_basket_y_pred_tfidf.todense()))

Accuracy (subset): 0.7816222348269994
Accuracy (ML): 0.9935370216850649
Precision (macro): 0.7376699073085848
Precision (micro): 0.8514760914760915
Recall (macro): 0.7288412123297471
Recall (micro): 0.8526815456362425
Hamming loss: 0.006462978314935207


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Word2Vec

In [17]:
big_basket_X_train_w2v = w2v_vectorize(big_basket_X_train)
big_basket_X_test_w2v = w2v_vectorize(big_basket_X_test)

In [18]:
ml_knn_w2v_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

ml_knn_w2v_grid_cv.fit(np.array([x for x in big_basket_X_train_w2v]), big_basket_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  23.4s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  22.5s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=  23.8s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=  24.5s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [19]:
ml_knn_w2v_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [20]:
ml_knn_w2v_best = MLkNN(**ml_knn_w2v_grid_cv.best_params_)
ml_knn_w2v_best.fit(np.array([x for x in big_basket_X_train_w2v]), big_basket_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [21]:
big_basket_y_pred_w2v = ml_knn_w2v_best.predict(np.array([x for x in big_basket_X_test_w2v]))

In [23]:
display_metrics(big_basket_y_test.to_numpy(), np.array(big_basket_y_pred_w2v.todense()))

Accuracy (subset): 0.742484401588202
Accuracy (ML): 0.9923589598150008
Precision (macro): 0.6923695251228656
Precision (micro): 0.8252896557472702
Recall (macro): 0.6897772201537334
Recall (micro): 0.8245336442371752
Hamming loss: 0.007641040184999346


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### BERT embeddings

In [25]:
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

In [26]:
big_basket_X_train_emb = sbert_model.encode(big_basket_X_train.values.tolist())
big_basket_X_test_emb = sbert_model.encode(big_basket_X_test.values.tolist())

In [28]:
ml_knn_emb_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

ml_knn_emb_grid_cv.fit(big_basket_X_train_emb, big_basket_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  24.6s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=  24.9s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=  24.5s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=  24.3s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [29]:
ml_knn_emb_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [30]:
ml_knn_emb_best = MLkNN(**ml_knn_w2v_grid_cv.best_params_)
ml_knn_emb_best.fit(big_basket_X_train_emb, big_basket_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [31]:
big_basket_y_pred_emb = ml_knn_emb_best.predict(big_basket_X_test_emb)

In [33]:
display_metrics(big_basket_y_test.to_numpy(), big_basket_y_pred_emb)

Accuracy (subset): 0.7647948572508981
Accuracy (ML): 0.9929716247073025
Precision (macro): 0.7194739236326118
Precision (micro): 0.83925
Recall (macro): 0.7110272512403599
Recall (micro): 0.838690872751499
Hamming loss: 0.007028375292697471


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Retail dataset

In [ ]:
retail_X_train, retail_X_test, retail_y_train, retail_y_test = train_test_split(
    retail_data['description'],
    retail_data[[column for column in retail_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13,
)

### TF-IDF

In [35]:
retail_X_train_tfidf, retail_X_test_tfidf = tfidf_vectorize(
    train_data=retail_X_train,
    test_data=retail_X_test,
)

In [36]:
retail_ml_knn_tfidf_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

retail_ml_knn_tfidf_grid_cv.fit(retail_X_train_tfidf, retail_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   6.8s
[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   6.7s
[CV 3/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   6.7s
[CV 1/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   6.5s
[CV 2/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   6.5s
[CV 3/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   6.5s
[CV 1/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   6.8s
[CV 2/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   6.9s
[CV 3/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   6.7s
[CV 1/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   7.0s
[CV 2/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   7.1s
[CV 3/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   7.1s
[CV 1/3] END .............k=

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [37]:
retail_ml_knn_tfidf_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [38]:
retail_ml_knn_tfidf_best = MLkNN(**retail_ml_knn_tfidf_grid_cv.best_params_)
retail_ml_knn_tfidf_best.fit(retail_X_train_tfidf, retail_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [41]:
retail_y_pred_tfidf = retail_ml_knn_tfidf_best.predict(retail_X_test_tfidf)

In [43]:
display_metrics(retail_y_test.to_numpy(), np.array(retail_y_pred_tfidf.todense()))

Accuracy (subset): 0.8318488529014845
Accuracy (ML): 0.9659801678108313
Precision (macro): 0.8306974479881335
Precision (micro): 0.8296555750145943
Recall (macro): 0.8307168608224904
Recall (micro): 0.8315001170138077
Hamming loss: 0.034019832189168576


### Word2Vec

In [45]:
retail_X_train_w2v = w2v_vectorize(retail_X_train)
retail_X_test_w2v = w2v_vectorize(retail_X_test)

In [46]:
retail_ml_knn_w2v_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

retail_ml_knn_w2v_grid_cv.fit(np.array([x for x in retail_X_train_w2v]), retail_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   4.9s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   4.9s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   5.0s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   5.0s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [47]:
retail_ml_knn_w2v_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [48]:
retail_ml_knn_w2v_best = MLkNN(**retail_ml_knn_w2v_grid_cv.best_params_)
retail_ml_knn_w2v_best.fit(np.array([x for x in retail_X_train_w2v]), retail_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [51]:
retail_y_pred_w2v = retail_ml_knn_w2v_best.predict(np.array([x for x in retail_X_test_w2v]))

In [52]:
display_metrics(retail_y_test.to_numpy(), np.array(retail_y_pred_w2v.todense()))

Accuracy (subset): 0.8226720647773279
Accuracy (ML): 0.9641260341489174
Precision (macro): 0.8265624390209665
Precision (micro): 0.8219354838709677
Recall (macro): 0.8175517040967598
Recall (micro): 0.8199157500585069
Hamming loss: 0.03587396585108256


### BERT embeddings

In [53]:
retail_sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

In [54]:
retail_X_train_emb = retail_sbert_model.encode(retail_X_train.values.tolist())
retail_X_test_emb = retail_sbert_model.encode(retail_X_test.values.tolist())

In [55]:
retail_ml_knn_emb_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

retail_ml_knn_emb_grid_cv.fit(retail_X_train_emb, retail_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   5.1s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   4.9s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   4.7s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   4.9s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [56]:
retail_ml_knn_emb_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [58]:
retail_ml_knn_emb_best = MLkNN(**retail_ml_knn_w2v_grid_cv.best_params_)
retail_ml_knn_emb_best.fit(retail_X_train_emb, retail_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [59]:
retail_y_pred_emb = retail_ml_knn_emb_best.predict(retail_X_test_emb)

In [60]:
display_metrics(retail_y_test.to_numpy(), retail_y_pred_emb)

Accuracy (subset): 0.8283400809716599
Accuracy (ML): 0.9656867922314146
Precision (macro): 0.8315168637183016
Precision (micro): 0.8300845467355566
Recall (macro): 0.8249676767342544
Recall (micro): 0.8271706061315235
Hamming loss: 0.03431320776858534


## Luxury dataset

In [61]:
luxury_X_train, luxury_X_test, luxury_y_train, luxury_y_test = train_test_split(
    luxury_data['description'],
    luxury_data[[column for column in luxury_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13,
)

### TF-IDF

In [62]:
luxury_X_train_tfidf, luxury_X_test_tfidf = tfidf_vectorize(
    train_data=luxury_X_train,
    test_data=luxury_X_test,
)

In [63]:
luxury_ml_knn_tfidf_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

luxury_ml_knn_tfidf_grid_cv.fit(luxury_X_train_tfidf, luxury_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   2.3s
[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   2.3s
[CV 3/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   2.3s
[CV 1/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   2.2s
[CV 2/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   2.3s
[CV 3/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time=   2.3s
[CV 1/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   2.2s
[CV 2/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   2.1s
[CV 3/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time=   2.2s
[CV 1/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   2.4s
[CV 2/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   2.3s
[CV 3/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time=   2.4s
[CV 1/3] END .............k=

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [64]:
luxury_ml_knn_tfidf_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [65]:
luxury_ml_knn_tfidf_best = MLkNN(**luxury_ml_knn_tfidf_grid_cv.best_params_)
luxury_ml_knn_tfidf_best.fit(luxury_X_train_tfidf, luxury_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [66]:
luxury_y_pred_tfidf = luxury_ml_knn_tfidf_best.predict(luxury_X_test_tfidf)

In [67]:
display_metrics(luxury_y_test.to_numpy(), np.array(luxury_y_pred_tfidf.todense()))

Accuracy (subset): 0.7159450897571278
Accuracy (ML): 0.9820710418117685
Precision (macro): 0.7002182951149984
Precision (micro): 0.7832512315270936
Recall (macro): 0.6941980270072069
Recall (micro): 0.7806873977086743
Hamming loss: 0.017928958188231595


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Word2Vec

In [68]:
luxury_X_train_w2v = w2v_vectorize(luxury_X_train)
luxury_X_test_w2v = w2v_vectorize(luxury_X_test)

In [69]:
luxury_ml_knn_w2v_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

luxury_ml_knn_w2v_grid_cv.fit(np.array([x for x in luxury_X_train_w2v]), luxury_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   1.9s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   1.9s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   2.1s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   2.1s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [70]:
luxury_ml_knn_w2v_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [71]:
luxury_ml_knn_w2v_best = MLkNN(**luxury_ml_knn_w2v_grid_cv.best_params_)
luxury_ml_knn_w2v_best.fit(np.array([x for x in luxury_X_train_w2v]), luxury_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [72]:
luxury_y_pred_w2v = luxury_ml_knn_w2v_best.predict(np.array([x for x in luxury_X_test_w2v]))

In [73]:
display_metrics(luxury_y_test.to_numpy(), np.array(luxury_y_pred_w2v.todense()))

Accuracy (subset): 0.7159450897571278
Accuracy (ML): 0.9823181828394258
Precision (macro): 0.6936269029897656
Precision (micro): 0.7873626373626373
Recall (macro): 0.6713727223854106
Recall (micro): 0.7817785051827605
Hamming loss: 0.017681817160574265


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### BERT Embeddings

In [74]:
luxury_sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

In [75]:
luxury_X_train_emb = luxury_sbert_model.encode(luxury_X_train.values.tolist())
luxury_X_test_emb = luxury_sbert_model.encode(luxury_X_test.values.tolist())

In [76]:
luxury_ml_knn_emb_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

luxury_ml_knn_emb_grid_cv.fit(luxury_X_train_emb, luxury_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   2.2s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time=   2.2s


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   2.2s


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time=   2.2s


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [77]:
luxury_ml_knn_emb_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [78]:
luxury_ml_knn_emb_best = MLkNN(**luxury_ml_knn_w2v_grid_cv.best_params_)
luxury_ml_knn_emb_best.fit(luxury_X_train_emb, luxury_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [79]:
luxury_y_pred_emb = luxury_ml_knn_emb_best.predict(luxury_X_test_emb)

In [80]:
display_metrics(luxury_y_test.to_numpy(), luxury_y_pred_emb)

Accuracy (subset): 0.7011615628299894
Accuracy (ML): 0.9805881956458247
Precision (macro): 0.6825968725869357
Precision (micro): 0.764898851831602
Recall (macro): 0.660657311400953
Recall (micro): 0.7632296781232951
Hamming loss: 0.01941180435417556


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Tech dataset

In [81]:
tech_X_train, tech_X_test, tech_y_train, tech_y_test = train_test_split(
    tech_data['description'],
    tech_data[[column for column in tech_data.columns if column != 'description']],
    test_size=0.2,
    random_state=13,
)

### TF-IDF

In [82]:
tech_X_train_tfidf, tech_X_test_tfidf = tfidf_vectorize(
    train_data=tech_X_train,
    test_data=tech_X_test,
)

In [83]:
tech_ml_knn_tfidf_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

tech_ml_knn_tfidf_grid_cv.fit(tech_X_train_tfidf, tech_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 3/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 1/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time= 1.6min
[CV 2/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time= 1.6min
[CV 3/3] END .............k=1, s=0.7; accuracy_ml: (test=nan) total time= 1.6min
[CV 1/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time= 1.6min
[CV 2/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time= 1.6min
[CV 3/3] END .............k=1, s=1.0; accuracy_ml: (test=nan) total time= 1.6min
[CV 1/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 2/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 3/3] END .............k=2, s=0.5; accuracy_ml: (test=nan) total time= 1.6min
[CV 1/3] END .............k=

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [84]:
tech_ml_knn_tfidf_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [85]:
tech_ml_knn_tfidf_best = MLkNN(**tech_ml_knn_tfidf_grid_cv.best_params_)
tech_ml_knn_tfidf_best.fit(tech_X_train_tfidf, tech_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [86]:
tech_y_pred_tfidf = tech_ml_knn_tfidf_best.predict(tech_X_test_tfidf)

In [87]:
display_metrics(tech_y_test.to_numpy(), np.array(tech_y_pred_tfidf.todense()))

Accuracy (subset): 0.8918025561921551
Accuracy (ML): 0.9990699777695541
Precision (macro): 0.8593072155313445


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision (micro): 0.918602861978766
Recall (macro): 0.8627480407472846


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall (micro): 0.9193101324299353
Hamming loss: 0.0009300222304459072


### Word2Vec

In [88]:
tech_X_train_w2v = w2v_vectorize(tech_X_train)
tech_X_test_w2v = w2v_vectorize(tech_X_test)

In [89]:
tech_ml_knn_w2v_grid_cv = GridSearchCV(
    MLkNN(),
    param_grid={
        'k': range(1,3),
        's': [0.5, 0.7, 1.0]
    },
    scoring={
        'accuracy_ml': make_scorer(accuracy_ml_score),
    },
    refit='accuracy_ml',
    cv=3,
    verbose=3,
)

tech_ml_knn_w2v_grid_cv.fit(np.array([x for x in tech_X_train_w2v]), tech_y_train.to_numpy())

Fitting 3 folds for each of 6 candidates, totalling 18 fits


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 1/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time= 1.6min


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=1, s=0.5; accuracy_ml: (test=nan) total time= 1.6min


...

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 2/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time= 1.6min


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/utils/_response.py", line 198, in _get_response_values
    classes = estimator.classes_
AttributeError: 

[CV 3/3] END .............k=2, s=1.0; accuracy_ml: (test=nan) total time= 1.7min


GridSearchCV(cv=3, estimator=MLkNN(),
             param_grid={'k': range(1, 3), 's': [0.5, 0.7, 1.0]},
             refit='accuracy_ml',
             scoring={'accuracy_ml': make_scorer(accuracy_ml_score, response_method='predict')},
             verbose=3)

In [90]:
tech_ml_knn_w2v_grid_cv.best_params_

{'k': 1, 's': 0.5}

In [91]:
tech_ml_knn_w2v_best = MLkNN(**tech_ml_knn_w2v_grid_cv.best_params_)
tech_ml_knn_w2v_best.fit(np.array([x for x in tech_X_train_w2v]), tech_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [92]:
tech_y_pred_w2v = tech_ml_knn_w2v_best.predict(np.array([x for x in tech_X_test_w2v]))

In [93]:
display_metrics(tech_y_test.to_numpy(), np.array(tech_y_pred_w2v.todense()))

Accuracy (subset): 0.8887174966945791
Accuracy (ML): 0.9989586930582187
Precision (macro): 0.8474461188225638


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision (micro): 0.9084691054411312
Recall (macro): 0.8564324252657364


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall (micro): 0.9101478287650139
Hamming loss: 0.001041306941781315


### BERT Embeddings

In [94]:
tech_sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

In [96]:
tech_X_train_emb = tech_sbert_model.encode(tech_X_train.values.tolist())
tech_X_test_emb = tech_sbert_model.encode(tech_X_test.values.tolist())

In [98]:
tech_ml_knn_emb_best = MLkNN(**luxury_ml_knn_w2v_grid_cv.best_params_)
tech_ml_knn_emb_best.fit(tech_X_train_emb, tech_y_train.to_numpy())

MLkNN(k=1, s=0.5)

In [99]:
tech_y_pred_emb = tech_ml_knn_emb_best.predict(tech_X_test_emb)

In [100]:
display_metrics(tech_y_test.to_numpy(), tech_y_pred_emb)

Accuracy (subset): 0.8913618334067871
Accuracy (ML): 0.9990390653497386
Precision (macro): 0.855330116138894
Precision (micro): 0.9153349723417332


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall (macro): 0.8596435650225405
Recall (micro): 0.9173082845703726
Hamming loss: 0.0009609346502612982
